На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [98]:
import pandas as pd
import numpy as np
from scipy import stats


In [99]:
df=pd.read_excel("AB_Test_Results.xlsx")
df

c:\Users\pavel\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
...,...,...,...
9995,1981,control,0.0
9996,502,variant,0.0
9997,9214,variant,0.0
9998,7741,control,0.0


In [100]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [102]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [103]:
df_new=df.astype({'USER_ID':np.uint32})

In [104]:
df_new.VARIANT_NAME.value_counts()

VARIANT_NAME
variant    5016
control    4984
Name: count, dtype: int64

In [105]:
df_new["purchase"]=np.where(df_new['REVENUE']>0, 1, 0)
df_new

,USER_ID,VARIANT_NAME,REVENUE,purchase
0,737,variant,0.0,0
1,2423,control,0.0,0
2,9411,control,0.0,0
3,7311,control,0.0,0
4,6174,variant,0.0,0
...,...,...,...,...
9995,1981,control,0.0,0
9996,502,variant,0.0,0
9997,9214,variant,0.0,0
9998,7741,control,0.0,0


In [106]:
df_new['USER_ID'].value_counts()

USER_ID
5652    6
8359    6
668     6
9101    6
4879    6
       ..
5455    1
1399    1
6967    1
3156    1
9468    1
Name: count, Length: 6324, dtype: int64

In [107]:
df_new=df_new.drop_duplicates()

In [108]:
df_new['USER_ID'].value_counts()

USER_ID
701     3
7358    3
9669    3
831     3
9942    3
       ..
4127    1
6398    1
9719    1
3812    1
9468    1
Name: count, Length: 6324, dtype: int64

In [109]:

s1 = pd.merge(df_new[df_new["VARIANT_NAME"]=="variant"], df_new[df_new["VARIANT_NAME"]=="control"], how='inner', on=['USER_ID'])
s1["USER_ID"]

0       6205
1       2529
2       2529
3       8406
4       9085
        ... 
1566    4034
1567    8471
1568    8471
1569    4883
1570    9303
Name: USER_ID, Length: 1571, dtype: uint32

In [110]:
df_new2 = df_new[df_new.USER_ID.isin (s1["USER_ID"]) == False ]
df_new2

,USER_ID,VARIANT_NAME,REVENUE,purchase
0,737,variant,0.0,0
4,6174,variant,0.0,0
5,2380,variant,0.0,0
7,9168,control,0.0,0
9,7548,control,0.0,0
...,...,...,...,...
9988,428,control,0.0,0
9991,8864,variant,0.0,0
9996,502,variant,0.0,0
9998,7741,control,0.0,0


In [111]:
df_new2.VARIANT_NAME.value_counts()

VARIANT_NAME
control    2414
variant    2407
Name: count, dtype: int64

In [112]:
df_new2.query('VARIANT_NAME == "control"').REVENUE.sum()

470.55999999999995

In [113]:
df_new2.query('VARIANT_NAME == "variant"').REVENUE.sum()

179.32

In [114]:
df_new2.query('VARIANT_NAME == "control"').purchase.sum()

54

In [115]:
df_new2.query('VARIANT_NAME == "variant"').purchase.sum()


43

In [116]:
df_new2.groupby('VARIANT_NAME')['REVENUE'].describe()

,count,mean,std,min,25%,50%,75%,max
VARIANT_NAME,,,,,,,,
control,2414.0,0.194930,4.151447,0.0,0.0,0.0,0.0,196.01
variant,2407.0,0.074499,0.854413,0.0,0.0,0.0,0.0,23.04


Промежуточный вывод:
Из среднего чека и общей суммы контрольной группы и тестируемой видно,что результаты в тестовой группе просаживаются по ключевым показателям, это говорит нам о том, что не стоит вводить тестовый вариант в продукт.
Далее проведем статистический анализ для этих двух групп, чтобы увидеть статистические различия.

In [117]:
my_stats1=stats.mannwhitneyu(x=df_new2[(df_new2['VARIANT_NAME'] == 'control')]['REVENUE'].values,
                                                  y=df_new2[(df_new2['VARIANT_NAME'] == 'variant')]['REVENUE'].values)
my_stats1

MannwhitneyuResult(statistic=2918468.0, pvalue=0.26069244720791673)

In [118]:
my_stats2=stats.mannwhitneyu(x=df_new2[(df_new2['VARIANT_NAME'] == 'control')]['purchase'].values,
                                                  y=df_new2[(df_new2['VARIANT_NAME'] == 'variant')]['purchase'].values)
my_stats2

MannwhitneyuResult(statistic=2918337.0, pvalue=0.2654139618736874)

Вывод:
 Данные в полученной таблице, с результатами А/Б тестирования, были собранны с ошибками, мы удалили много дубликатов, многие пользователи участвовали в двух группах(также были удалены), на основание исправленного дата фрейма мы можем сделать вывод, что значимых изменений в конверсии не наблюдается и обновление можно не выкатывать.

Проведем тоже самое на первых табличных данных, чтобы увидеть на сколько данные могут разниться.

In [119]:
my_stats3=stats.mannwhitneyu(x=df[(df['VARIANT_NAME'] == 'control')]['REVENUE'].values,
                                                  y=df[(df['VARIANT_NAME'] == 'variant')]['REVENUE'].values)
my_stats3

MannwhitneyuResult(statistic=12521564.0, pvalue=0.47825247965294926)